<h1>Creation of phenopackets from tabular data (individuals in columns)</h1>
<p>We will process <a href="https://pubmed.ncbi.nlm.nih.gov/16783569/" target="__blank">Budny, et al. (2006) A novel X-linked recessive mental retardation syndrome comprising macrocephaly and ciliary dysfunction is allelic to oral-facial-digital type I syndrome</a></p> in this notebook.
<p>pyphetools provides a convenient way of extracting HPO terms from typical tables presented in supplemental material. Typical tables can have the individuals in columns or rows. In this case, we extract data from TABLE.</p>
<p>This note shows how to work through the table and set up the pyphetools encoder. The table was not originally available in the table, but constructed using the data in the publication</p>
<p>Users can work on one column at a time and then generate a collection of <a href="https://pubmed.ncbi.nlm.nih.gov/35705716/" target="__blank">GA4GH phenopackets</a> to represent each patient included in the original supplemental material. These phenopackets can then be used for a variety of downstream applications.</p>

In [1]:
import phenopackets as php
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
import os
import sys
import numpy as np
from pyphetools.creation import *
from pyphetools.creation.simple_column_mapper import try_mapping_columns, get_separate_hpos_from_df
# Last tested with pyphetools 0.4.5

<h2>Importing HPO data</h2>
<p>pyphetools uses the Human Phenotype Ontology (HPO) to encode phenotypic features. The recommended way of doing this is to ingest the hp.json file using HpoParser, which in turn creates an HpoConceptRecognizer object. </p>
<p>The HpoParser can accept a hpo_json_file argument if you want to use a specific file. If the argument is not passed, it will download the latext hp.json file from the HPO GitHub site and store it in a new subdirectory called hpo_data. It will not download the file if the file is already downloaded.</p>

In [2]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
metadata = MetaData(created_by="ORCID:0000-0002-5648-2155")
metadata.default_versions_with_hpo(version=hpo_version)

<h2>Importing the supplemental table</h2>
<p>Here, we use the pandas library to import this file (note that the Python package called openpyxl must be installed to read Excel files with pandas, although the library does not need to be imported in this notebook). pyphetools expects a pandas DataFrame as input, and users can choose any input format available for pandas include CSV, TSV, and Excel, or can use any other method to transform their input data into a Pandas DataFrame before using pyphetools.</p>

In [3]:
df = pd.read_excel('input/PMID_16783569.xlsx')

In [4]:
df

,individual,sex,age,variant,Developmental delay,Abnormal respiratory system physiology,Macrocephaly,Recurrent respiratory infections,"Intellectual disability, severe",High palate,Low-set ears,Broad thumb,Brachydactyly,Obesity,Large for gestational age,Postaxial polydactyly,Inguinal hernia
0,1,male,11,c.2122_2125dupGAAG,+,+,+,+,+,+,+,+,+,+,-,NaN,NaN
1,2,male,0,c.2122_2125dupGAAG,+,+,+,+,+,NaN,NaN,NaN,NaN,NaN,+,+,NaN
2,3,male,0,c.2122_2125dupGAAG,+,+,+,+,NaN,NaN,NaN,NaN,+,NaN,NaN,NaN,+
3,4,male,0,c.2122_2125dupGAAG,+,+,+,+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,male,0,c.2122_2125dupGAAG,+,+,+,+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,male,0,c.2122_2125dupGAAG,+,+,+,+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,male,0,c.2122_2125dupGAAG,+,+,+,+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,male,0,c.2122_2125dupGAAG,+,+,+,+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,male,0,c.2122_2125dupGAAG,+,+,+,+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h2>Column mappers</h2>
<p>Please see the notebook "Create phenopackets from tabular data with individuals in rows" for explanations. In the following cell we create a dictionary for the ColumnMappers. Note that the code is identical except that we use the df.loc function to get the corresponding row data</p>

In [5]:
hpo_cr = parser.get_hpo_concept_recognizer()
column_mapper_d, col_not_found = try_mapping_columns(df=df,
                                                    observed='+',
                                                    excluded='-',
                                                    hpo_cr=hpo_cr,
                                                    preview=True)

                                      term    status
0  Global developmental delay (HP:0001263)  observed
1  Global developmental delay (HP:0001263)  observed
2  Global developmental delay (HP:0001263)  observed
3  Global developmental delay (HP:0001263)  observed
4  Global developmental delay (HP:0001263)  observed
5  Global developmental delay (HP:0001263)  observed
6  Global developmental delay (HP:0001263)  observed
7  Global developmental delay (HP:0001263)  observed
8  Global developmental delay (HP:0001263)  observed
                                                  term    status
0  Abnormal respiratory system physiology (HP:0002795)  observed
1  Abnormal respiratory system physiology (HP:0002795)  observed
2  Abnormal respiratory system physiology (HP:0002795)  observed
3  Abnormal respiratory system physiology (HP:0002795)  observed
4  Abnormal respiratory system physiology (HP:0002795)  observed
5  Abnormal respiratory system physiology (HP:0002795)  observed
6  Abnormal res

In [6]:
print(col_not_found)        

['individual', 'sex', 'age', 'variant']


<h2>Variant Data</h2>
<p>The variant data (HGVS< transcript) is listed in the Variant (hg19, NM_015133.4) column.</p>

In [7]:
genome = 'hg19'
default_genotype = 'hemizygous'
transcript='NM_003611.3'
varMapper = VariantColumnMapper(assembly=genome,column_name='variant', 
                                transcript=transcript, default_genotype=default_genotype)

<h1>Demographic data</h1>
<p>pyphetools can be used to capture information about age, sex, and individual identifiers. This information is stored in a map of "IndividualMapper" objects. Special treatment may be required for the indifiers, which may be used as the column names or row index.</p>

In [8]:
ageMapper = AgeColumnMapper.by_year('age')
ageMapper.preview_column(df['age'])

,original column contents,age
0,11,P11Y
1,0,P0Y


In [9]:
sexMapper = SexColumnMapper(male_symbol='male', female_symbol='female', column_name='sex')
sexMapper.preview_column(df['sex'])

,original column contents,sex
0,male,MALE
1,male,MALE
2,male,MALE
3,male,MALE
4,male,MALE
5,male,MALE
6,male,MALE
7,male,MALE
8,male,MALE


In [10]:
pmid = "PMID:16783569"
encoder = CohortEncoder(df=df, hpo_cr=hpo_cr, column_mapper_d=column_mapper_d, 
                        individual_column_name="individual", agemapper=ageMapper, sexmapper=sexMapper,
                       variant_mapper=varMapper, metadata=metadata,
                       pmid=pmid)
encoder.set_disease(disease_id='300209', label='Simpson-Golabi-Behmel syndrome, type 2')

In [11]:
individuals = encoder.get_individuals()

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_003611.3%3Ac.2122_2125dupGAAG/NM_003611.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_003611.3%3Ac.2122_2125dupGAAG/NM_003611.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_003611.3%3Ac.2122_2125dupGAAG/NM_003611.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_003611.3%3Ac.2122_2125dupGAAG/NM_003611.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_003611.3%3Ac.2122_2125dupGAAG/NM_003611.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_003611.3%3Ac.2122_2125dupGAAG/NM_003611.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_003611.3%3Ac.2122_2125du

In [12]:
i1 = individuals[-1]
phenopacket1 = i1.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh())
json_string = MessageToJson(phenopacket1)
print(json_string)

{
  "id": "9",
  "subject": {
    "id": "9",
    "timeAtLastEncounter": {
      "age": {
        "iso8601duration": "P0Y"
      }
    },
    "sex": "MALE"
  },
  "phenotypicFeatures": [
    {
      "type": {
        "id": "HP:0001263",
        "label": "Global developmental delay"
      }
    },
    {
      "type": {
        "id": "HP:0002795",
        "label": "Abnormal respiratory system physiology"
      }
    },
    {
      "type": {
        "id": "HP:0000256",
        "label": "Macrocephaly"
      }
    },
    {
      "type": {
        "id": "HP:0002205",
        "label": "Recurrent respiratory infections"
      }
    }
  ],
  "interpretations": [
    {
      "id": "9",
      "progressStatus": "SOLVED",
      "diagnosis": {
        "disease": {
          "id": "300209",
          "label": "Simpson-Golabi-Behmel syndrome, type 2"
        },
        "genomicInterpretations": [
          {
            "subjectOrBiosampleId": "9",
            "interpretationStatus": "CAUSATIVE",
     

In [13]:
output_directory = "../../phenopackets/OFD1/"
encoder.output_phenopackets(outdir=output_directory)

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_003611.3%3Ac.2122_2125dupGAAG/NM_003611.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_003611.3%3Ac.2122_2125dupGAAG/NM_003611.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_003611.3%3Ac.2122_2125dupGAAG/NM_003611.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_003611.3%3Ac.2122_2125dupGAAG/NM_003611.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_003611.3%3Ac.2122_2125dupGAAG/NM_003611.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_003611.3%3Ac.2122_2125dupGAAG/NM_003611.3?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg19/NM_003611.3%3Ac.2122_2125du